In [15]:
from aocd import get_data

puzzle_input = get_data(day=12, year=2023)

In [1]:
it1 = """
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
""".strip()

Partie 1

In [8]:
def parse_input(input: str):
    records = []
    for row in input.split("\n"):
        record, group_counts = row.split(" ")
        records.append((record, tuple(int(i) for i in group_counts.split(","))))
    return records

parse_input(it1)

[('???.###', (1, 1, 3)),
 ('.??..??...?##.', (1, 1, 3)),
 ('?#?#?#?#?#?#?#?', (1, 3, 1, 6)),
 ('????.#...#...', (4, 1, 1)),
 ('????.######..#####.', (1, 6, 5)),
 ('?###????????', (3, 2, 1))]

In [10]:
def naive_count_matchs(record: str, group_counts: tuple[int]) -> int:
    nb_matchs = 0

    completes = []
    no_completes = [record]
    while no_completes:
        current:str = no_completes.pop()
        try:
            ind = current.index("?")
            no_completes.append(f"{current[:ind]}.{current[ind+1:]}")
            no_completes.append(f"{current[:ind]}#{current[ind+1:]}")
        except ValueError:
            completes.append(current)

    for comp in completes:
        groups_lengths = tuple(len(g) for g in comp.split(".") if g)
        nb_matchs += groups_lengths == group_counts

    return nb_matchs


def naive_count_arrangements(records):
    return sum([naive_count_matchs(record, group_counts) for record, group_counts in records])

naive_count_arrangements(parse_input(it1))

21

In [16]:
naive_count_arrangements(parse_input(puzzle_input))

7718

Partie 2